In [1]:
from vggish_smoke_test import *

/home/roxor/bin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters



Testing your install of VGGish



KeyboardInterrupt: 

In [1]:
import vggish_input

In [2]:
from scipy.io import wavfile

In [3]:
a = wavfile.read("test.wav")

In [4]:
a

(48000, array([[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]], dtype=int16))

In [5]:
a = wavfile.read("mine.wav")

In [6]:
a

(44100, array([[0, 0],
        [0, 0],
        [0, 0],
        ...,
        [0, 0],
        [0, 0],
        [0, 0]], dtype=int16))

In [7]:
samples = a[1] / 32768.0 

In [8]:
samples[21334]

array([-0.00076294, -0.00076294])

In [9]:
import vggish_params

In [10]:
vggish_params.SAMPLE_RATE

16000

In [11]:
vggish_params.STFT_HOP_LENGTH_SECONDS

0.01

In [12]:
features_sample_rate = 1.0 / vggish_params.STFT_HOP_LENGTH_SECONDS
example_window_length = int(round(
      vggish_params.EXAMPLE_WINDOW_SECONDS * features_sample_rate))
example_hop_length = int(round(
      vggish_params.EXAMPLE_HOP_SECONDS * features_sample_rate))

In [13]:
example_hop_length

96

In [14]:
example_window_length

96

In [15]:
a = vggish_input.wavfile_to_examples("test.wav")

In [41]:
# Test these new functions with the original test.
import tensorflow as tf
tf.reset_default_graph()
sess = tf.Session()

vgg = CreateVGGishNetwork(0.01)

# Generate a 1 kHz sine wave at 44.1 kHz (we use a high sampling rate
# to test resampling to 16 kHz during feature extraction).
num_secs = 3
freq = 1000
sr = 44100
t = np.linspace(0, num_secs, int(num_secs * sr))
x = np.sin(2 * np.pi * freq * t)  # Unit amplitude input signal


INFO:tensorflow:Restoring parameters from vggish_model.ckpt


In [43]:
x.shape

(132300,)

In [15]:
b = vggish_input.wavfile_to_examples("2.wav")

In [16]:
b.shape

(308, 96, 64)

In [52]:
mine = vggish_input.wavfile_to_examples("test.wav")

In [53]:
mine.shape

(283, 96, 64)

In [19]:
import vggish_slim
import vggish_params
import vggish_input
import vggish_postprocess

def CreateVGGishNetwork(hop_size=0.96):   # Hop size is in seconds.
  """Define VGGish model, load the checkpoint, and return a dictionary that points
  to the different tensors defined by the model.
  """
  vggish_slim.define_vggish_slim()
  checkpoint_path = 'vggish_model.ckpt'
  vggish_params.EXAMPLE_HOP_SECONDS = hop_size
  
  vggish_slim.load_vggish_slim_checkpoint(sess, checkpoint_path)

  features_tensor = sess.graph.get_tensor_by_name(
      vggish_params.INPUT_TENSOR_NAME)
  embedding_tensor = sess.graph.get_tensor_by_name(
      vggish_params.OUTPUT_TENSOR_NAME)

  layers = {'conv1': 'vggish/conv1/Relu',
            'pool1': 'vggish/pool1/MaxPool',
            'conv2': 'vggish/conv2/Relu',
            'pool2': 'vggish/pool2/MaxPool',
            'conv3': 'vggish/conv3/conv3_2/Relu',
            'pool3': 'vggish/pool3/MaxPool',
            'conv4': 'vggish/conv4/conv4_2/Relu',
            'pool4': 'vggish/pool4/MaxPool',
            'fc1': 'vggish/fc1/fc1_2/Relu',
            'fc2': 'vggish/fc2/Relu',
            'embedding': 'vggish/embedding',
            'features': 'vggish/input_features',
         }
  g = tf.get_default_graph()
  for k in layers:
    layers[k] = g.get_tensor_by_name( layers[k] + ':0')
    
  return {'features': features_tensor,
          'embedding': embedding_tensor,
          'layers': layers,
         }

/home/roxor/bin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [20]:
def ProcessWithVGGish(vgg, x, sr):
  '''Run the VGGish model, starting with a sound (x) at sample rate
  (sr). Return a whitened version of the embeddings. Sound must be scaled to be
  floats between -1 and +1.'''

  # Produce a batch of log mel spectrogram examples.
  input_batch = vggish_input.waveform_to_examples(x, sr)
  # print('Log Mel Spectrogram example: ', input_batch[0])

  [embedding_batch] = sess.run([vgg['embedding']],
                               feed_dict={vgg['features']: input_batch})
  return embedding_batch
  # Postprocess the results to produce whitened quantized embeddings.
  pca_params_path = 'vggish_pca_params.npz'

  pproc = vggish_postprocess.Postprocessor(pca_params_path)
  postprocessed_batch = pproc.postprocess(embedding_batch)
  # print('Postprocessed VGGish embedding: ', postprocessed_batch[0])
  return postprocessed_batch[0]


In [54]:
import tensorflow as tf
tf.reset_default_graph()
sess = tf.Session()

vgg = CreateVGGishNetwork(1)

INFO:tensorflow:Restoring parameters from vggish_model.ckpt


In [16]:
import numpy as np
sr, wav_data = wavfile.read("2.wav")
assert wav_data.dtype == np.int16, 'Bad sample type: %r' % wav_data.dtype
samples = wav_data / 32768.0  # Convert to [-1.0, +1.0]

In [18]:
len(samples)

14208000

In [59]:
values = ProcessWithVGGish(vgg,samples,sr)

In [60]:
values.shape

(296, 128)

In [49]:
len(values)

5

In [45]:
values2 = values

In [38]:
values2[0]-values[0]

7

In [51]:
values2.shape

(308, 128)

In [22]:
import six
num_secs = 5
freq = 1000
sr = 44100
t = np.linspace(0, num_secs, int(num_secs * sr))
x = np.sin(2 * np.pi * freq * t)
# Convert to signed 16-bit samples.
samples = np.clip(x * 32768, -32768, 32767).astype(np.int16)
wav_file = six.BytesIO()
wavfile.write(wav_file, sr, samples)
wav_file.seek(0)

0

In [23]:
examples_batch = vggish_input.wavfile_to_examples(wav_file)
print(examples_batch)

[[[-4.47771674 -4.29076017 -4.1532819  ... -3.98410919 -3.93030654
   -3.76891238]
  [-4.487119   -4.28663482 -4.14313869 ... -3.98851528 -3.9346542
   -3.78824514]
  [-4.460583   -4.29542372 -4.14568167 ... -3.97235414 -3.9172474
   -3.79094514]
  ...
  [-4.45912554 -4.29392363 -4.14827892 ... -3.9534104  -3.94283579
   -3.78986623]
  [-4.45680333 -4.29347445 -4.15260834 ... -3.96181455 -3.92302184
   -3.7812338 ]
  [-4.46679676 -4.28966581 -4.15162289 ... -3.95698292 -3.94577437
   -3.77947337]]

 [[-4.46342334 -4.29210809 -4.1494392  ... -3.96689471 -3.94103314
   -3.78345147]
  [-4.46518298 -4.29021104 -4.14963754 ... -3.9627394  -3.92718488
   -3.78323181]
  [-4.46232002 -4.29156065 -4.15015574 ... -3.96022267 -3.94609914
   -3.79095732]
  ...
  [-4.47229232 -4.29333028 -4.15550377 ... -3.96550264 -3.92026994
   -3.78052026]
  [-4.47493192 -4.2918865  -4.15142573 ... -3.98084516 -3.93156007
   -3.7891335 ]
  [-4.47940576 -4.29223346 -4.14987884 ... -3.96548313 -3.91986638
   -3.79

In [24]:
examples_batch.shape

(5, 96, 64)

In [25]:
example_hop_length = int(round(
        vggish_params.EXAMPLE_HOP_SECONDS * features_sample_rate))

In [26]:
example_hop_length

96

In [27]:
example_hop_length = int(round(
        0.04 * features_sample_rate))

In [28]:
example_hop_length

4

In [29]:
vggish_params.EXAMPLE_HOP_SECONDS

0.96